In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,BatchNormalization,LayerNormalization
from keras.layers import Conv1D,Conv2D
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Flatten
from keras.layers import Dropout,Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn import preprocessing
import keras
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from keras import regularizers
from sklearn.preprocessing import QuantileTransformer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train_features=pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
df_train_features_targets=pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

df_train_features=df_train_features.iloc[:,1:]
df_train_features_targets=df_train_features_targets.iloc[:,1:]



#encoding all the catergorical data
cp_type_encodded = preprocessing.LabelEncoder()
df_train_features['cp_type']=cp_type_encodded.fit_transform(df_train_features['cp_type'].tolist())

cp_dose_encdded = preprocessing.LabelEncoder()
df_train_features['cp_dose']=cp_dose_encdded.fit_transform(df_train_features['cp_dose'].tolist())

# print(df_train_features)
# print(df_train_features_targets)






scaler=MinMaxScaler()
df_train_features_scaled=scaler.fit_transform(df_train_features)

qt = QuantileTransformer(n_quantiles=20, random_state=0)
df_train_features_scaled=qt.fit_transform(df_train_features_scaled)

pca1 = PCA(0.95)
df_train_features_scaled=pca1.fit_transform(df_train_features_scaled)

# poly = PolynomialFeatures(2)
# df_train_features_scaled=poly.fit_transform(df_train_features)




# X_train, X_test, y_train, y_test = train_test_split(df_train_features_scaled, df_train_features_targets, test_size=0.33, random_state=42)

# from sklearn.decomposition import PCA


In [ ]:
print(df_train_features_scaled.shape)

In [ ]:


print(df_train_features_scaled)
print(df_train_features_targets.values)

target_values=df_train_features_targets.values

from sklearn.model_selection import KFold

kf = KFold(n_splits=5)
accuracy_list=[]
count=0
for train, test in kf.split(df_train_features_scaled, target_values):
    print(train)
    print(df_train_features_scaled[train])
    print(target_values[train])
    print(df_train_features_scaled[train].shape)
    model =Sequential()

#     model.add(Dense(2048, activation="relu",input_shape=(128,)))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.2))
#     model.add(Dense(1024, activation="relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.4))
#     model.add(Dense(512, activation="relu"))
#     model.add(BatchNormalization())
#     model.add(Dropout(0.4))
#     model.add(Dense(206, activation="relu"))
#     opt = keras.optimizers.Adam(learning_rate=0.001)          
#     model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])


    model.add(Dense(1024, activation="relu",input_shape=(702,),kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(512, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(256, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(206, activation="relu"))
    opt = keras.optimizers.Adam(learning_rate=0.001)          
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    batch_size=256
    epochs=500
    history=model.fit(df_train_features_scaled[train], target_values[train], batch_size=batch_size, epochs=epochs,validation_data=(df_train_features_scaled[test], target_values[test]),verbose=2)
    y_pred=model.predict(df_train_features_scaled[test])
    y_pred = (y_pred > 0.5) 
    print(y_pred)
    print(accuracy_score(target_values[test], y_pred))
    accuracy_list.append(accuracy_score(target_values[test], y_pred))
    model_name='delta_'+str(count)+'.h5'
    model.save(model_name)
    count=count+1
    
    

In [ ]:

index_lication=accuracy_list.index(max(accuracy_list))


model_name='delta_'+str(index_lication)+'.h5'
model= keras.models.load_model(model_name)



df_test_features=pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sample_submission=pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


df_test_features=df_test_features.iloc[:,1:]
# df_train_features_targets=df_train_features_targets.iloc[:,1:]



#encoding all the catergorical data
# cp_type_encodded = preprocessing.LabelEncoder()
df_test_features['cp_type']=cp_type_encodded.fit_transform(df_test_features['cp_type'].tolist())

# cp_dose_encdded = preprocessing.LabelEncoder()
df_test_features['cp_dose']=cp_dose_encdded.fit_transform(df_test_features['cp_dose'].tolist())

# print(df_train_features)
# print(df_train_features_targets)


# scaler=MinMaxScaler()
df_test_features_scaled=scaler.transform(df_test_features)
df_test_features_scaled=qt.transform(df_test_features_scaled)
df_test_features_scaled=pca1.transform(df_test_features_scaled)


prediction_result=model.predict(df_test_features_scaled)
print(prediction_result)

column_list=list(sample_submission.columns)
column_list=column_list[1:]

submission_df=pd.DataFrame(data=prediction_result,columns=column_list)



submission_df.insert(0,'sig_id', sample_submission['sig_id'].tolist())

print(submission_df)

submission_df.to_csv('submission.csv',index=False)